In [ ]:
import os
from glob import glob

# 读取路径
image_sequence_path = "/Users/WangHao/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/佳文数据_0722/短轴视频_0722"
image_sequence_list = os.listdir(image_sequence_path)
image_sequence_list = sorted(image_sequence_list)

for path_name in image_sequence_list:
    # 读取文件
    ext = "npy"
    file_list = glob(f"{os.path.join(image_sequence_path, path_name)}/*.{ext}")
    try:
        for file in file_list:
            os.remove(file)
    except:
        print(f"{path_name}中不存在{ext}类型文件")

print("运行结束")

In [5]:
import os
import shutil

path = r"/Users/WangHao/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/王昊数据_0801/"
save_path = "/Users/WangHao/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/dataset_0802"

# 通过DFS实现统一逻辑处理同一层次的文件对象
for root, dirs, files in os.walk(path):
    if not dirs:
        keywords = ["label", "visual"]
        dir_name = root.split("/")[-1]
        keys = [key for key in keywords if key in dir_name]
        if not keys:
            save_image_path = os.path.join(save_path, "image", dir_name)
            if not os.path.exists(save_image_path):
                os.makedirs(save_image_path)
            ext = ".png"
            for file_name in files:
                if ext in file_name:
                    shutil.copy(os.path.join(root, file_name), save_image_path)

        if keys == ["label"]:
            save_label_path = os.path.join(save_path, "label", dir_name.replace("_label", ""))
            if not os.path.exists(save_label_path):
                os.makedirs(save_label_path)
            ext = ".png"
            for file_name in files:
                if ext in file_name:
                    shutil.copy(os.path.join(root, file_name), save_label_path)

print("运行结束")

运行结束


In [ ]:
import cv2
import numpy as np

filePath = '/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset_0802/label/06570820220302_CA MASHAOYONG_20220302091109144'
if not os.path.exists(filePath.replace('dataset_0706', 'dataset_0706_nb')):
    os.mkdir(filePath.replace('dataset_0706', 'dataset_0706_nb'))
for root, dir, files in os.walk(filePath):
    for i in files:
        if 'label' in i:
            label = cv2.imdecode(np.fromfile(f"{root}/{i}", dtype=np.uint8), -1)
            label[..., 0] = 0
            image = cv2.imread(root + '/' + i.replace('label', 'image'))

            if not os.path.exists(
                    root.replace('dataset_0706', 'dataset_0706_nb')):
                os.mkdir(root.replace('dataset_0706', 'dataset_0706_nb'))

            cv2.imencode('.png', label)[-1].tofile(
                root.replace('dataset_0706', 'dataset_0706_nb') + '/' + i)
            cv2.imencode('.png', image)[-1].tofile(
                root.replace('dataset_0706', 'dataset_0706_nb') + '/' +
                i.replace('label', 'image'))

print("运行结束")

In [ ]:
import numpy as np
import os
import cv2
from skimage import morphology


def remove_isolate(inputs, threshold_area=0.5):
    """
    inputs: x*y*3
    """
    mask = np.zeros((inputs.shape[0], inputs.shape[1]), dtype=np.uint8)
    mask[np.sum(inputs, axis=-1) > 0] = 1
    mask = morphology.remove_small_objects(mask.astype(np.bool8),
                                           np.sum(mask) * threshold_area,
                                           connectivity=8).astype(np.uint8)
    outputs = np.expand_dims(mask, -1) * inputs

    return outputs


def crop_array(image, label, up, down, left, right):
    '''
    image:x*y*3 uint8
    label:x*y*3 uint8
    '''
    crop_image = image[up:down, left:right, :]
    crop_label = label[up:down, left:right, :]

    return crop_image, crop_label


def count_place(base_path, file_list):
    place = []
    for file_name in file_list:
        data = cv2.imdecode(
            np.fromfile(f"{base_path}/{file_name}", dtype=np.uint8), -1)
        data_gray = cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)[0]
        x_sum = np.sum(data_gray, axis=0)
        y_sum = np.sum(data_gray, axis=1)

        left = min(np.where(x_sum != 0)) - 20 if min(np.where(x_sum != 0)) > 20 else 0
        right = max(np.where(x_sum != 0)) + 20 if min(np.where(x_sum != 0)) < data.shape[0] - 20 else data.shape[0]
        up = min(np.where(y_sum != 0)) - 20 if min(np.where(y_sum != 0)) > 20 else 0
        down = max(np.where(y_sum != 0)) + 20 if max(np.where(y_sum != 0)) < data.shape[1] - 20 else data.shape[1]

    place.append([up, down, left, right])
    
    return np.array(place)


file_path = '/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset_0802'
save_path = file_path.replace('dataset_0802', 'dataset_roi_0802')
if not os.path.exists(save_path):
    os.mkdir(save_path)
for dir_path in os.listdir(file_path):
    if dir_path == 'label':
        label = cv2.imread(root + '/' + i)
        image = cv2.imread(root + '/' + i.replace('label', 'image'))
        image = cv2.resize(image.astype(np.uint8), (384, 320),
                            cv2.INTER_CUBIC)
        label = cv2.resize(label.astype(np.uint8), (384, 320),
                            cv2.INTER_NEAREST)
        crop_image, crop_label = crop_array(image, label)
        if not os.path.exists(
                root.replace('dataset0706', 'data_get_roi_0706')):
            os.mkdir(root.replace('dataset0706', 'data_get_roi_0706'))
        cv2.imencode('.png', crop_label)[-1].tofile(
            root.replace('dataset0706', 'data_get_roi_0706') + '/' + i)
        cv2.imencode('.png', crop_image)[-1].tofile(
            root.replace('dataset0706', 'data_get_roi_0706') + '/' +
            i.replace('label', 'image'))

print("运行结束")